# Show some statistics of the enclosed geometric object with psydat file

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from psychopy.misc import fromFile
from vstt.stats import get_velocity

### Import
A psydat file can be imported using the psychopy `fromFile` function: 
If you want to know the detailed content of the data in psydat file, please check the notebook 'raw_data.ipynb'

In [ ]:
psydata = fromFile("example.psydat")

### Plot of results for each trial
For example, a scatter plot of the mouse positions for each trial, labelled by the condition, trial number and repetition number:

In [ ]:
colors = ["blue", "green", "red", "cyan", "magenta", "yellow", "black", "orange"]

nTrials, nReps = psydata.sequenceIndices.shape
fig, axs = plt.subplots(nTrials, nReps, figsize=(6, 6 * nTrials * nReps))
axs = np.reshape(
    axs, (nTrials, nReps)
)  # ensure axs is a 2d-array even if nTrials or nReps is 1
for trial in range(nTrials):
    for rep in range(nReps):
        loc = (trial, rep)
        condition = psydata.sequenceIndices[loc]
        target_radius = psydata.trialList[condition]["target_size"]
        central_target_radius = psydata.trialList[condition]["central_target_size"]
        ax = axs[loc]
        ax.set_title(f"Trial {trial}, Rep {rep} [Condition {condition}]")
        for positions, target_pos, color in zip(
            psydata.data["to_target_mouse_positions"][loc],
            psydata.data["target_pos"][loc],
            colors,
        ):
            ax.plot(positions[:, 0], positions[:, 1], color=color)
            ax.add_patch(
                plt.Circle(
                    target_pos,
                    target_radius,
                    edgecolor="none",
                    facecolor=color,
                    alpha=0.1,
                )
            )
        if not psydata.trialList[condition]["automove_cursor_to_center"]:
            for positions, color in zip(
                psydata.data["to_center_mouse_positions"][loc],
                colors,
            ):
                ax.plot(positions[:, 0], positions[:, 1], color=color)
            ax.add_patch(
                plt.Circle(
                    [0, 0],
                    central_target_radius,
                    edgecolor="none",
                    facecolor="black",
                    alpha=0.1,
                )
            )

plt.show()

### Plot of results for each target
For example, a scatter plot of the mouse positions for each target, labelled by trial number, repetition number, target number and condition:

In [ ]:
fig, axs = plt.subplots(nTrials, nReps * 8, figsize=(6 * 8, 6 * nTrials * nReps))
axs = np.reshape(
    axs, (nTrials, nReps * 8)
)  # ensure axs is a 2d-array even if nTrials or nReps is 1
for trial in range(nTrials):
    for rep in range(nReps):
        loc = (trial, rep)
        condition = psydata.sequenceIndices[loc]
        for positions, color, i in zip(
            psydata.data["to_target_mouse_positions"][loc],
            colors,
            range(8),
        ):
            ax = axs[(trial, rep + i)]
            ax.set_title(
                f"Trial {trial}, Rep {rep}, Target {i} [Condition {condition}]"
            )
            ax.set_xlim(-0.5, 0.5)
            ax.set_ylim(-0.5, 0.5)
            if not psydata.trialList[condition]["automove_cursor_to_center"]:
                positions = np.concatenate(
                    (positions, psydata.data["to_center_mouse_positions"][loc][i]),
                    axis=0,
                )
            ax.plot(positions[:, 0], positions[:, 1], color=color)

fig.delaxes(axs[2][6])
fig.delaxes(axs[2][7])
fig.delaxes(axs[3][6])
fig.delaxes(axs[3][7])
plt.show()

### Plot velocity for each trial
For example, a scatter plot of the velocity for each target, labelled by trial number, repetition number and condition:

In [ ]:
fig, axs = plt.subplots(nTrials, nReps, figsize=(6, 6 * nTrials * nReps))
axs = np.reshape(
    axs, (nTrials, nReps)
)  # ensure axs is a 2d-array even if nTrials or nReps is 1
for trial in range(nTrials):
    for rep in range(nReps):
        loc = (trial, rep)
        condition = psydata.sequenceIndices[loc]
        ax = axs[loc]
        ax.set_title(f"Trial {trial}, Rep {rep} [Condition {condition}]")

        final_position = []
        final_timesteps = []
        for positions, timestamps, i in zip(
            psydata.data["to_target_mouse_positions"][loc],
            psydata.data["to_target_timestamps"][loc],
            range(8),
        ):
            final_position.append(positions)
            # print(f'only to target positions : {final_position}')
            final_timesteps.append(timestamps)
            if not psydata.trialList[condition]["automove_cursor_to_center"]:
                final_position.append(psydata.data["to_center_mouse_positions"][loc][i])
                # print(f'add to center positions : {final_position}')
                final_timesteps.append(psydata.data["to_center_timestamps"][loc][i])
        ax.plot(
            np.concatenate(final_timesteps)[:-1],
            get_velocity(
                np.concatenate(final_timesteps), np.concatenate(final_position)
            ),
        )

plt.show()

In [ ]:
fig, axs = plt.subplots(nTrials, nReps * 8, figsize=(6 * 8, 6 * nTrials * nReps))
axs = np.reshape(
    axs, (nTrials, nReps * 8)
)  # ensure axs is a 2d-array even if nTrials or nReps is 1
for trial in range(nTrials):
    for rep in range(nReps):
        loc = (trial, rep)
        condition = psydata.sequenceIndices[loc]
        for positions, timestamps, color, i in zip(
            psydata.data["to_target_mouse_positions"][loc],
            psydata.data["to_target_timestamps"][loc],
            colors,
            range(8),
        ):
            ax = axs[(trial, rep + i)]
            ax.set_title(
                f"Trial {trial}, Rep {rep}, Target {i} [Condition {condition}]"
            )
            if not psydata.trialList[condition]["automove_cursor_to_center"]:
                positions = np.concatenate(
                    (positions, psydata.data["to_center_mouse_positions"][loc][i]),
                    axis=0,
                )
                timestamps = np.concatenate(
                    (timestamps, psydata.data["to_center_timestamps"][loc][i])
                )
            ax.plot(timestamps[:-1], get_velocity(timestamps, positions), color=color)
fig.delaxes(axs[2][6])
fig.delaxes(axs[2][7])
fig.delaxes(axs[3][6])
fig.delaxes(axs[3][7])
plt.show()

In [ ]:
fig, axs = plt.subplots(nTrials, nReps, figsize=(6, 6 * nTrials * nReps))
axs = np.reshape(
    axs, (nTrials, nReps)
)  # ensure axs is a 2d-array even if nTrials or nReps is 1
for trial in range(nTrials):
    for rep in range(nReps):
        loc = (trial, rep)
        condition = psydata.sequenceIndices[loc]
        ax = axs[loc]
        ax.set_title(f"Trial {trial}, Rep {rep} [Condition {condition}]")
        for positions, timestamps, color, i in zip(
            psydata.data["to_target_mouse_positions"][loc],
            psydata.data["to_target_timestamps"][loc],
            colors,
            range(8),
        ):
            if not psydata.trialList[condition]["automove_cursor_to_center"]:
                positions = np.concatenate(
                    (positions, psydata.data["to_center_mouse_positions"][loc][i]),
                    axis=0,
                )
                timestamps = np.concatenate(
                    (timestamps, psydata.data["to_center_timestamps"][loc][i])
                )
            ax.plot(
                timestamps[:-1] - timestamps[0],
                get_velocity(timestamps, positions),
                color=color,
            )
plt.show()